In [1]:
import json
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
# import boto3
from datetime import datetime

from dotenv import load_dotenv

import pandas as pd

In [2]:
load_dotenv()


client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

scope = "user-read-recently-played"

True

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Define the desired scopes as a list
scopes = ["user-read-recently-played", "user-library-read"]

# Initialize the SpotifyOAuth object with your client ID, client secret, and desired scopes
sp_oauth = SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri='http://localhost:8888/callback',
    scope=' '.join(scopes)
)

# # # Generate the authorization URL
# auth_url = sp_oauth.get_authorize_url()

# # Redirect the user to the generated auth_url to start the authorization process
# print(f"Click the following link to authorize your application: {auth_url}")

sp = spotipy.Spotify(client_credentials_manager = sp_oauth)

Click the following link to authorize your application: https://accounts.spotify.com/authorize?client_id=20c0bbcdf29241b39fd44c8a912827ad&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&scope=user-read-recently-played+user-library-read


In [11]:
# GET the recently played tracks
recently_played = sp.current_user_recently_played()

In [12]:
## Save the json to df

song_names = []
artist_names = []
played_at_list = []
timestamps = []

# Extracting only the relevant bits of data from the json object      
for song in recently_played["items"]:
    song_names.append(song["track"]["name"])
    artist_names.append(song["track"]["album"]["artists"][0]["name"])
    played_at_list.append(song["played_at"])
    timestamps.append(song["played_at"][0:10])
    
# Prepare a dictionary in order to turn it into a pandas dataframe below       
song_dict = {
    "song_name" : song_names,
    "artist_name": artist_names,
    "played_at" : played_at_list,
    "timestamp" : timestamps
}

song_df = pd.DataFrame(song_dict, columns = ["song_name", "artist_name", "played_at", "timestamp"])

In [13]:
song_df.head()

,song_name,artist_name,played_at,timestamp
0,This Moment,DVBBS,2023-10-15T23:42:39.220Z,2023-10-15
1,Strangers,Kenya Grace,2023-10-15T23:40:23.343Z,2023-10-15
2,More Baby,Chris Lake,2023-10-15T23:37:45.929Z,2023-10-15
3,Atmosphere,FISHER,2023-10-15T23:34:03.921Z,2023-10-15
4,Real Love,Martin Garrix,2023-10-15T23:31:04.684Z,2023-10-15


In [14]:
#Extract playlist data

playlists = sp.user_playlists('spotify')
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXcDYGt49X0ozW"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]

spotify_data = sp.playlist_tracks(playlist_URI)   

In [16]:
import sqlite3

conn = sqlite3.connect('spotify.db')
cursor = conn.cursor()

# # Get the path to the database file
# cursor.execute("PRAGMA database_list;")
# database_path = cursor.fetchone()[2]

# print("Database path:", database_path)

# Use the to_sql method to write the DataFrame to a new table in the database
table_name = 'recently_played'  # Name for the new table in the database
song_df.to_sql(table_name, conn, if_exists='replace', index=False)

# Close the database connection
conn.close()

Database path: /Users/yusali/dev/ETL_Spotify/notebook/spotify.db


In [18]:
conn = sqlite3.connect('spotify.db')
df = pd.read_sql('select * from {}'.format(table_name), conn)
conn.close()

In [25]:
df.to_parquet('../data/recently_payed.parquet', partition_cols=['timestamp', 'artist_name'])